Market depth (order book)
==============

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('20.230.202.72', 7497, clientId=550058)

<IB connected to 20.230.202.72:7497 clientId=550058>

To get a list of all exchanges that support market depth data and display the first five:

In [2]:
l = ib.reqMktDepthExchanges()
l[:5]

[DepthMktDataDescription(exchange='DTB', secType='OPT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='COMEX', secType='FOP', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='SGX', secType='FUT', listingExch='', serviceDataType='Deep', aggGroup=2147483647),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', listingExch='', serviceDataType='Deep', aggGroup=4)]

Let's subscribe to market depth data for EURUSD:

In [3]:
# contract = Forex('EURUSD')
# contract = Future(symbol="NQ", localSymbol="NQU3", exchange="CME")
contract = Future(symbol="ES", localSymbol="ESU3", exchange="CME")
ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)

To see a live order book, an event handler for ticker updates is made that displays a dynamically updated dataframe:

In [4]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(10),
        columns='bidSize bidPrice askPrice askSize'.split())

def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(10):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(10):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)

ticker.updateEvent += onTickerUpdate

IB.sleep(15);

,bidSize,bidPrice,askPrice,askSize
0,31.0,4585.75,4586.0,39.0
1,69.0,4585.5,4586.25,52.0
2,75.0,4585.25,4586.5,44.0
3,90.0,4585.0,4586.75,89.0
4,65.0,4584.75,4587.0,62.0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


Stop the market depth subscription:

In [5]:
ib.cancelMktDepth(contract)

In [6]:
ib.disconnect()